### **INF3831 - Bases de Datos**
# Evaluación 1: SQL

## 1. Introducción
---
En esta tarea evaluaremos sus conocimientos de SQL.

En este notebook dejaremos el espacio para escribir sus consultas, y probar si funcionan correctamente.

Para entregar su solución, basta con escribir sus respuestas en el notebook, **guardar los cambios**, descargar el notebook, y subirlo por canvas.







## 2. Esquema

Para esta actividad vamos a trabajar con datos de una empresa que vende productos a través de varias tiendas. Trabajaremos con el siguiente esquema:

- Tiendas(tid PRIMARY KEY, calle, comuna, region)
- Productos(pid PRIMARY KEY, nombre, precio, tipo)
- Vende(tid,pid) PRIMARY KEY(tid,pid)
- Usuarios(uid PRIMARY KEY, nombre, calle, comuna, region)
- Compras(cid PRIMARY KEY, uid, tid)
- Contenido_Compra(cid,pid,cantidad) PRIMARY KEY (cid,pid)

En este esquema, la tabla 'Tiendas' tiene información sobre cada tienda de la empresa. La información que guardaremos sobre cada tienda es su id único, y la dirección de la tienda, compuesta por calle, comuna y región. El repositorio (único) de productos se guarda en la tabla 'Productos', donde cada producto se describe a través de su id, nombre, su precio y tipo. Para registrar que una tienda en particular vende un producto, se ocupa la tabla 'Vende', vinculando la tienda con el producto. Por otro lado, la tabla 'Usuarios' tiene un registro de los usuarios de la empresa, respaldando la información con su nombre y dirección.

Para registrar las compras que los usuarios realizan de una tienda específica, se tienen dos tablas. La primera, llamada 'Compras' establece una compra de un usuario en una tienda, y la tabla 'Contenido_Compra' guarda la información sobre los productos comprados en cada compra individual. También se guarda la información sobre la cantidad de unidades de cada producto comprado para poder modelar que un usuario compra varias unidades del mismo producto en una compra.

## 3. Datos

Aquí crearemos una instancia del esquema de arriba e insertaremos algunos datos a la instancia.

La idea es que cada tabla debería tener al menos una fila y cada consulta de abajo debería devolver al menos una respuesta con los datos especificados.

In [2]:
# pin SQLAlchemy version for colab compatibility
!pip install SQLAlchemy==1.4.46

# Si dejan de funcionar las cosas descomentar estas dos lineas:
#!pip install PyMySQL==1.0.2 # aqui deberia ser psql
#!pip install ipython-sql==0.4.1

# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [975 kB]
Hit:9 http://ppa.launchp

In [3]:
%%sql
DROP TABLE IF EXISTS Tiendas;
CREATE TABLE Tiendas(tid INT PRIMARY KEY, calle VARCHAR(200), comuna VARCHAR(100), region VARCHAR(200));
INSERT INTO Tiendas VALUES(1, 'Andres Bello 111', 'Providencia', 'RM');
INSERT INTO Tiendas VALUES(2, 'Avenida Argentina 125', 'Valparaíso','Valparaíso');
INSERT INTO Tiendas VALUES(3, 'Calle 10 Norte 145B', 'Viña del Mar', 'Valparaíso');

DROP TABLE IF EXISTS Productos;
CREATE TABLE Productos(pid INT PRIMARY KEY, nombre VARCHAR(200), precio FLOAT, tipo VARCHAR(100));
INSERT INTO Productos VALUES(100, 'Polera BTS', 40, 'Ropa');
INSERT INTO Productos VALUES(200, 'Polera RTS', 20, 'Ropa');
INSERT INTO Productos VALUES(300, 'Refrigerador', 1000, 'Electro');
INSERT INTO Productos VALUES(400, 'Mancuerna 10kgs', 80, 'Deporte');
INSERT INTO Productos VALUES(500, 'Banda elástica', 10, 'Deporte');
INSERT INTO Productos VALUES(600, 'Pelota Futbol', 200, 'Deporte');
INSERT INTO Productos VALUES(700, 'Cafetera', 100, 'Electro');
INSERT INTO Productos VALUES(800, 'Monitor', 200, 'Tecnología');
INSERT INTO Productos VALUES(900, 'Notebook', 1200, 'Tecnología');
INSERT INTO Productos VALUES(1000, 'Lomo kg', 10, 'Carne');

DROP TABLE IF EXISTS Vende;
CREATE TABLE Vende(tid INT, pid INT, PRIMARY KEY(tid, pid));
INSERT INTO Vende VALUES(1, 100);
INSERT INTO Vende VALUES(1, 200);
INSERT INTO Vende VALUES(1, 300);
INSERT INTO Vende VALUES(1, 400);
INSERT INTO Vende VALUES(1, 500);
INSERT INTO Vende VALUES(1, 600);
INSERT INTO Vende VALUES(1, 700);
INSERT INTO Vende VALUES(1, 800);
INSERT INTO Vende VALUES(1, 900);
INSERT INTO Vende VALUES(2, 100);
INSERT INTO Vende VALUES(2, 200);
INSERT INTO Vende VALUES(2, 300);
INSERT INTO Vende VALUES(2, 700);
INSERT INTO Vende VALUES(3, 100);
INSERT INTO Vende VALUES(3, 200);
INSERT INTO Vende VALUES(3, 300);
INSERT INTO Vende VALUES(3, 400);
INSERT INTO Vende VALUES(3, 500);
INSERT INTO Vende VALUES(3, 700);
INSERT INTO Vende VALUES(3, 800);
INSERT INTO Vende VALUES(3, 900);
INSERT INTO Vende VALUES(1, 1000);

DROP TABLE IF EXISTS Usuarios;
CREATE TABLE Usuarios(uid INT PRIMARY KEY, nombre VARCHAR(200), calle VARCHAR(200), comuna VARCHAR(100), region VARCHAR(200));
INSERT INTO Usuarios VALUES(1, 'Valentina', 'Calle 123', 'Conchalí', 'RM');
INSERT INTO Usuarios VALUES(2, 'Adrián', 'Dirección 145', 'Maipú', 'RM');
INSERT INTO Usuarios VALUES(5, 'Lomos', 'Dirección 90', 'Maipú', 'RM');
INSERT INTO Usuarios VALUES(3, 'Conito', 'Otra Calle 78', 'Viña del Mar', 'Valparaíso');
INSERT INTO Usuarios VALUES(4, 'Frambuesa', 'Más direcciones 90', 'Concón', 'Valparaíso');
INSERT INTO Usuarios VALUES(22, 'Domagoj', 'VicunaMackenn 4860', 'Macul', 'RM');


DROP TABLE IF EXISTS Compras;
CREATE TABLE Compras(cid INT PRIMARY KEY, uid INT, tid INT);
INSERT INTO Compras VALUES(1, 1, 1);
INSERT INTO Compras VALUES(101, 1, 1);
INSERT INTO Compras VALUES(3, 1, 1);
INSERT INTO Compras VALUES(4, 1, 2);
INSERT INTO Compras VALUES(5, 1, 2);
INSERT INTO Compras VALUES(6, 2, 1);
INSERT INTO Compras VALUES(7, 2, 2);
INSERT INTO Compras VALUES(8, 2, 3);
INSERT INTO Compras VALUES(9, 3, 3);
INSERT INTO Compras VALUES(10, 3, 3);
INSERT INTO Compras VALUES(11, 3, 1);
INSERT INTO Compras VALUES(12, 4, 1);

DROP TABLE IF EXISTS Contenido_Compra;
CREATE TABLE Contenido_Compra(cid INT, pid INT, cantidad INT, PRIMARY KEY(cid, pid));
INSERT INTO Contenido_Compra VALUES(1, 100, 2);
INSERT INTO Contenido_Compra VALUES(101, 800, 2);
INSERT INTO Contenido_Compra VALUES(101, 900, 1);
INSERT INTO Contenido_Compra VALUES(101, 100, 1);
INSERT INTO Contenido_Compra VALUES(101, 1000, 3);
INSERT INTO Contenido_Compra VALUES(3, 400, 2);
INSERT INTO Contenido_Compra VALUES(4, 200, 3);
INSERT INTO Contenido_Compra VALUES(5, 100, 1);
INSERT INTO Contenido_Compra VALUES(6, 100, 3);
INSERT INTO Contenido_Compra VALUES(6, 600, 1);
INSERT INTO Contenido_Compra VALUES(7, 300, 1);
INSERT INTO Contenido_Compra VALUES(7, 700, 1);
INSERT INTO Contenido_Compra VALUES(8, 200, 4);
INSERT INTO Contenido_Compra VALUES(8, 1000, 2);
INSERT INTO Contenido_Compra VALUES(9, 100, 2);
INSERT INTO Contenido_Compra VALUES(10, 100, 1);
INSERT INTO Contenido_Compra VALUES(11, 200, 1);
INSERT INTO Contenido_Compra VALUES(12, 500, 3);

 * postgresql+psycopg2://@/postgres


""


In [4]:
%%sql
SELECT *
FROM Tiendas;

 * postgresql+psycopg2://@/postgres


,tid,calle,comuna,region
0,1,Andres Bello 111,Providencia,RM
1,2,Avenida Argentina 125,Valparaíso,Valparaíso
2,3,Calle 10 Norte 145B,Viña del Mar,Valparaíso


In [5]:
%%sql
SELECT *
FROM Usuarios;

 * postgresql+psycopg2://@/postgres


,uid,nombre,calle,comuna,region
0,1,Valentina,Calle 123,Conchalí,RM
1,2,Adrián,Dirección 145,Maipú,RM
2,5,Lomos,Dirección 90,Maipú,RM
3,3,Conito,Otra Calle 78,Viña del Mar,Valparaíso
4,4,Frambuesa,Más direcciones 90,Concón,Valparaíso
5,22,Domagoj,VicunaMackenn 4860,Macul,RM


In [6]:
%%sql
SELECT *
FROM Contenido_Compra;

 * postgresql+psycopg2://@/postgres


,cid,pid,cantidad
0,1,100,2
1,101,800,2
2,101,900,1
3,101,100,1
4,101,1000,3
5,3,400,2
6,4,200,3
7,5,100,1
8,6,100,3
9,6,600,1


## 4. Consultas [6 puntos]

Escriban las siguentes consultas SQL que funcionan sobre el esquema de arriba. Cada consulta vale 0.5 puntos.


**1**. Liste el precio del producto más carro.

In [7]:
%%sql
SELECT * 
FROM Productos as p1
WHERE p1.precio = 
  (SELECT MAX(p2.precio)
  FROM Productos AS p2 );

 * postgresql+psycopg2://@/postgres


,pid,nombre,precio,tipo
0,900,Notebook,1200.0,Tecnología


2. Liste el número de usuarios viviendo en la Región Metropolitana.

In [8]:
%%sql
SELECT COUNT(*)
FROM Usuarios
WHERE region = 'RM'
GROUP BY region;

 * postgresql+psycopg2://@/postgres


,count
0,4


3. Liste nombre y precio de todos los productos vendidos por una tienda ubicada en la región 'Vaparaíso' (cuidado con acentos). Los productos deben ser ordenado por precio, desde más carro hasta el más barato. La lista de respuestas no debería contener productos repetidos.

In [9]:
%%sql
SELECT DISTINCT Productos.nombre,  Productos.precio
FROM Productos, Vende, Tiendas
WHERE Productos.pid = Vende.pid and Tiendas.tid = Vende.tid and Tiendas.region = 'Valparaíso'
ORDER BY Productos.precio DESC;

 * postgresql+psycopg2://@/postgres


,nombre,precio
0,Notebook,1200.0
1,Refrigerador,1000.0
2,Monitor,200.0
3,Cafetera,100.0
4,Mancuerna 10kgs,80.0
5,Polera BTS,40.0
6,Polera RTS,20.0
7,Banda elástica,10.0


4. Para cada tienda, liste el número de productos vendidos por esta tienda.

In [10]:
%%sql
SELECT tid, COUNT(DISTINCT pid) AS Numero_productos
FROM Vende
GROUP BY tid;

 * postgresql+psycopg2://@/postgres


,tid,numero_productos
0,1,10
1,2,4
2,3,8


5. Retornar nombre del usuario, junto con el *precio* del producto más barato que compró. Si el usuario no realizó ninguna compra, retornar 0:

In [18]:
%%sql
SELECT DISTINCT Usuarios.nombre, t1.preciomin
FROM Usuarios
LEFT JOIN (
  SELECT MIN(Productos.precio) AS preciomin, Compras.uid FROM Productos, Compras, Contenido_Compra
  WHERE Productos.pid = Contenido_Compra.pid AND Compras.cid = Contenido_Compra.cid
  GROUP BY Compras.uid
  UNION
  SELECT DISTINCT 0 as preciomin, Usuarios.uid
  FROM Usuarios, Compras
  WHERE Usuarios.uid NOT IN (SELECT Compras.uid FROM compras)
  ) AS t1 ON (Usuarios.uid = t1.uid)
;

 * postgresql+psycopg2://@/postgres


,nombre,preciomin
0,Adrián,10.0
1,Conito,20.0
2,Domagoj,0.0
3,Frambuesa,10.0
4,Lomos,0.0
5,Valentina,10.0


6. Retornar nombres de los usuarios que realizaron la compra con el mayor valor: 

In [11]:
%%sql
SELECT Usuarios.nombre
FROM Productos, Compras, Contenido_Compra, Usuarios
WHERE Productos.pid = Contenido_Compra.pid 
AND Compras.cid = Contenido_Compra.cid 
AND Usuarios.uid = Compras.uid 
AND Productos.precio IN (
  SELECT Max(p1.precio )
  FROM Productos as p1, Compras, Contenido_Compra
  WHERE p1.pid = Contenido_Compra.pid 
    AND Compras.cid = Contenido_Compra.cid)


 * postgresql+psycopg2://@/postgres


,nombre
0,Valentina


7. Para cada región, listar el id de la tienda más preferida por los usuarios que **no** viven en esta región. Quiere decir, para cada tienda hay que computar el número de usuarios distintos que viven en una región distinta a la región dónde se ubica la tienda, y quienes realizaron una compra en dicha tienda. Se considera el número de usuarios **distintos**; quiere decir, si un usuario realizó dos compras, esto suma 1 al conteo de usuarios. Después para cada región detectar la tienda con mayor número de usuarios computados de esta manera.

In [12]:
%%sql
SELECT agg.region, MAX(agg.conteo)
FROM (
  SELECT Tiendas.tid, Tiendas.region, COUNT(DISTINCT Usuarios.uid) as conteo
  FROM Usuarios, Compras, Tiendas
  WHERE Usuarios.region <> Tiendas.region AND Tiendas.tid = Compras.tid AND  Compras.uid = Usuarios.uid 
  GROUP BY Tiendas.tid, Tiendas.region
  ) as agg
GROUP BY region;


 * postgresql+psycopg2://@/postgres


,region,max
0,RM,2
1,Valparaíso,2


8. Listar nombre de los usuarios que compraron uno de los cinco productos más caros. Para sacar los cinco más caros, usar LIMIT:

In [13]:
%%sql
SELECT DISTINCT Usuarios.nombre
FROM Usuarios, Productos AS p0, Compras, Contenido_Compra
WHERE Usuarios.uid = Compras.uid 
  AND Compras.cid = Contenido_Compra.cid 
  AND p0.pid = Contenido_Compra.pid 
  AND p0.pid IN
  (SELECT p1.pid
    FROM Productos as p1
    ORDER BY precio DESC
    LIMIT 5)

 * postgresql+psycopg2://@/postgres


,nombre
0,Adrián
1,Valentina


9. Listar Id de cada tienda, junto con el promedio de los precios de todos los productos que vende esta tienda

In [14]:
%%sql
SELECT Tiendas.tid, AVG(Productos.precio) AS promedio
FROM Tiendas, Vende, Productos
WHERE
  Tiendas.tid = Vende.tid AND Productos.pid = Vende.pid
GROUP BY Tiendas.tid;


 * postgresql+psycopg2://@/postgres


,tid,promedio
0,2,290.00
1,3,331.25
2,1,286.00


10. Listar nombres de todos los usuarios que realizaron una compra dónde compraron tres o más productos distintos (si el mismo producto fue comprado 3 veces o mas, esto cuenta cómo un producto).

In [44]:
%%sql
SELECT Usuarios.nombre
FROM Usuarios, Compras, Contenido_Compra
WHERE Usuarios.uid = Compras.uid AND Contenido_compra.cid = Compras.cid
GROUP BY Usuarios.uid, Contenido_Compra.cid
HAVING COUNT(DISTINCT Contenido_Compra.pid) >= 3
;


 * postgresql+psycopg2://@/postgres


,nombre
0,Valentina


11. Para cada compra con **tres o más productos**, listar su id, el nombre del usuario, y el número de productos comprados en esta compra. Si un producto fue comprado dos o más veces, hay que contar esto (quiere decir: aquí sí cuentan los productos repetidos):

In [16]:
%%sql
SELECT Compras.cid, Usuarios.nombre, sum(Contenido_Compra.cantidad)
FROM Usuarios, Compras, Contenido_Compra
WHERE Usuarios.uid = Compras.uid and Contenido_compra.cid = Compras.cid
GROUP BY Compras.cid, Usuarios.nombre
HAVING Sum(Contenido_Compra.cantidad) >= 3;

 * postgresql+psycopg2://@/postgres


,cid,nombre,sum
0,8,Adrián,6
1,6,Adrián,4
2,4,Valentina,3
3,12,Frambuesa,3
4,101,Valentina,7


12. Liste los nombres de los productos más baratos (puede haber más qué uno).

In [17]:
%%sql
SELECT DISTINCT Productos.nombre
FROM Productos 
WHERE Productos.precio = ANY (SELECT MIN(Productos.precio) FROM Productos)


 * postgresql+psycopg2://@/postgres


,nombre
0,Banda elástica
1,Lomo kg
